# Technical recap 26th May2020

## Our previous approaches for the GP-system identification model for reactor dynamics. 
The core of the GP-bases system idenfitication is the assumption that the dynamics is following the state space model as below
$$
x_{t+1} = f(x_t,u_t) + v_t,\\
y_t = g(x_t,u_t) + w_t
$$
The GP enters the *SSM* by assuming that
$$
f(x_t,u_t) \sim {GP} (m_f(x_t,u_t),k_f(x_t,u_t,x_t',u_t')),\\
g(x_t,u_t) \sim {GP} (m_g(x_t,u_t),k_g(x_t,u_t,x_t',u_t')),\\
x_0 \sim p(x_0), \\
f_t := f(x_{t-1}),\\
x_t|f(x_{t-1}) \sim  \mathcal{N}(0,Q),\\
g_t := g(x_t),\\
y_t|g_t \sim \mathcal{N}(0,R).
$$

### One idea, is to lump the dynamics of the observation as a state as given in Frigola, 2015 (this [reference](https://www.repository.cam.ac.uk/handle/1810/299413)).
- That is now, we only have GP for the transition function $f$, where the new latent state $z$ are the original latent states and the observations
- The observations are some of the last elements in $z_t$
$$
f(z_t,u_t) \sim {GP} (m_f(z_t,u_t),k_f(z_t,u_t,z_t',u_t')),\\
z_0 \sim p(x_0), \\
f_t := f(z_{t-1}),\\
z_t|f(z_{t-1}) \sim  \mathcal{N}(0,Q),\\
$$

- Now, we can use $m_f$ as a linear dynamics with the structure
 $$
 m_f(z_t,u_t) = A_0 z_t + B_0 u_t.
 $$
 - The matrices $A$ and $B$ are obtained from linear system identification procedure
 - Using finite dimensional approximation of the GP of $f$, we write
 $$
 f \sim A_0 z_t + B_0 u_t + A \Phi(z_t, u_t)
 $$
 - Here, $\Phi$ is a basis function, with the dimension is $n_z \times n_b^{n_z+n_u}$.  
 - The matrix $A$ to be sampled by some MCMC, also the covariance matrix $Q$.
 
## Result

!["Example of Delta T"](image/delta_T_0.png)


- After several trial and errors with the parameter of the simulations, the simulation result is still very bad.
- One of the possible problem is that lumping the observation into the latent state.
- Other possible bottle neck is that the basis function $\Phi$ dimension grows exponentially with the number of state $n_z$. 
- Selecting a higher state than $4$ is not feasible using basis number $5$.
- While at the same time the dynamic dimensions it self probably more than that.
- Also incorporation of the catalyst activity level in the model (ODE, PDE) is missing. (Comments from Feedback) 

## Our second approach:   The PDE model (Price and Butt, 1977)

\begin{equation}
\begin{split}
  \frac{\partial s}{\partial t}
  &= D_s \, \frac{\partial^2 s}{\partial z^2}
  - C_s \, \frac{\partial a}{\partial z}
  - K_1 \, r_s(s,T), c \\
  \frac{\partial a}{\partial t}
  &= D_a \, \frac{\partial^2 a}{\partial z^2}
  - C_a \, \frac{\partial a}{\partial z}
  - K_2 \, r_a(a,c,T), c \\
  \frac{\partial T}{\partial t}
  &= D_T \, \frac{\partial^2 T}{\partial z^2}
  - C_T \, \frac{\partial T}{\partial z}
  + K_3 \, r_a(a,c,T), c\\
  \frac{\partial c}{\partial t}
  &= K_5 \, r_s(s,T) \, c,
\end{split} \label{eqs:PDE_butt_price}
\end{equation}

where $s = s(t,z)$, $a = a(t,z)$, $T = T(t,z)$, and $c = c(t,z)$. Also, note that $C_s$ = $C_a$ and $C_T = constant* C_s$, $K_1$, $K_2$ and $K_3$ are correlated. The boundary and initial conditions for the partial differential equations above are:

\begin{align}
    T(z=0) =& T_0,&\; \left.\dfrac{\partial T}{\partial z}\right|_{z=L} =& 0.\\
    a(z=0) =& a_0,&\; \left.\dfrac{\partial a}{\partial z}\right|_{z=L} =& 0,\\
    s(z=0) =& s_0,&\; \left.\dfrac{\partial s}{\partial z}\right|_{z=L} =& 0,\\
    c(t=0) =& 1. & &.
\end{align}

Further we have

\begin{equation}
  r_a(a,c,T) = -d_1 \, \exp(-d_2 / T) \, a \, c
\end{equation}

After simplification we arrive at

\begin{align}
\hat{m}^-(t,\omega_z) =& F(\theta) \hat{m}(t-1,\omega_z),\\
P^-(t,\theta) =& F(\theta) P(t,\theta) F(\theta)^\top + G(\theta) G(\theta)^\top,\\
\tilde{y}(t) =& y(t) -  H \hat{m}^-(t,\omega_z),\\
S(t,\theta) =& H P^-(t,\theta) H^\top + R(\theta),\\
K =& P^-(t,\theta)H^\top S(t,\theta)^{-1},\\
\hat{m}(t,\omega_z) =& \hat{m}^-(t,\omega_z) + \tilde{y}(t),\\
P(t,\theta) =& P^-(t,\theta) - P^-(t,\theta)S(t,\theta)^{-1}P^-(t,\theta).
\end{align}

Now the problem has become a parameter estimation problem.

## Next step

#### 1. We are implementing the parameter estimation part
#### 2. We also have to evaluate this approach in more details
 

### Reference
1. Frigola, R. (2015). Bayesian Time Series Learning with Gaussian Processes (Doctoral thesis). https://doi.org/10.17863/CAM.46480
2. Thomas H. Price, John B. Butt, Catalyst poisoning and fixed bed reactor dynamics—II: Adiabatic reactors, Chemical Engineering Science, Volume 32, Issue 4, 1977, Pages 393-412, ISSN 0009-2509,https://doi.org/10.1016/0009-2509(77)85006-9.